In [2]:
import kagglehub

/opt/anaconda3/envs/emailspam/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("zeeshanyounas001/email-spam-detection")

print("Path to dataset files:", path)

100%|██████████| 418k/418k [00:00<00:00, 4.60MB/s]

Extracting files...
Path to dataset files: /Users/suweilin/.cache/kagglehub/datasets/zeeshanyounas001/email-spam-detection/versions/2


In [4]:
! mv /Users/suweilin/.cache/kagglehub/datasets/zeeshanyounas001/email-spam-detection/versions/2 ./data

In [5]:
import pandas as pd
import torch
import numpy as np


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/opt/anaconda3/envs/emailspam/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/opt/anaconda3/envs/emailspam/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/opt/anaconda3/envs/emailspam/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 739, in start
    self.io_loop.start(

In [7]:
df = pd.read_csv('data/spam mail.csv')

In [8]:
df

,Category,Masseges
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...
